In [10]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [11]:
path_train = "UJIndoorLoc/trainingData.csv"
path_validation = "UJIndoorLoc/validationData.csv"

In [12]:
#Explicitly pass header=0 to be able to replace existing names 
train_df = pd.read_csv(path_train,header = 0)
train_df = train_df[:19930]
train_AP_strengths = train_df.ix[:,:520] #select first 520 columns

#Scale transforms data to center to the mean and component wise scale to unit variance
train_AP_features = scale(np.asarray(train_AP_strengths))

#The following two objects are actually pandas.core.series.Series objects
building_ids_str = train_df["BUILDINGID"].map(str) #convert all the building ids to strings
building_floors_str = train_df["FLOOR"].map(str) #convert all the building floors to strings

res = building_ids_str + building_floors_str #element wise concatenation of BUILDINGID+FLOOR
train_labels = np.asarray(building_ids_str + building_floors_str)

#convert labels to categorical variables, dummy_labels has type 'pandas.core.frame.DataFrame'
dummy_labels = pd.get_dummies(train_labels)


"""one hot encode the dummy_labels.
this is done because dummy_labels is a dataframe with the labels (BUILDINGID+FLOOR) 
as the column names
"""
train_labels = np.asarray(dummy_labels) #labels is an array of shape 19937 x 13. (there are 13 types of labels)

In [13]:
#generate len(train_AP_features) of floats in between 0 and 1
train_val_split = np.random.rand(len(train_AP_features))
#convert train_val_split to an array of booleans: if elem < 0.7 = true, else: false
train_val_split = train_val_split < 0.70 #should contain ~70% percent true

We aren't given a formal testing set, so we will treat the given validation set as the testing set

In [14]:
# We will then split our given training set into training + validation 
train_X = train_AP_features[train_val_split]
train_y = train_labels[train_val_split]
val_X = train_AP_features[~train_val_split]
val_y = train_labels[~train_val_split]

In [15]:
#Turn the given validation set into a testing set
test_df = pd.read_csv(path_validation,header = 0)
test_AP_features = scale(np.asarray(test_df.ix[:,0:520]))
test_labels = np.asarray(test_df["BUILDINGID"].map(str) + test_df["FLOOR"].map(str))
test_labels = np.asarray(pd.get_dummies(test_labels))

In [16]:
nb_epochs = 20
batch_size = 10
input_size = 520
num_classes = 13

In [17]:
def encoder():
    model = Sequential()
    model.add(Dense(256, input_dim=input_size, activation='tanh', bias=True))
    model.add(Dense(128, activation='tanh', bias=True))
    model.add(Dense(64, activation='tanh', bias=True))
    return model

In [18]:
def decoder(e):   
    e.add(Dense(128, input_dim=64, activation='tanh', bias=True))
    e.add(Dense(256, activation='tanh', bias=True))
    e.add(Dense(input_size, activation='tanh', bias=True))
    e.compile(optimizer='adam', loss='mse')
    return e
    

In [19]:
e = encoder()

In [20]:
d = decoder(e)

In [21]:
d.fit(train_X, train_X, nb_epoch=nb_epochs, batch_size=batch_size)

Epoch 1/20
13827/13827 [==============================] - 12s - loss: 0.7052    
Epoch 2/20
13827/13827 [==============================] - 12s - loss: 0.6817    
Epoch 3/20
13827/13827 [==============================] - 12s - loss: 0.6759    
Epoch 4/20
13827/13827 [==============================] - 13s - loss: 0.6727    
Epoch 5/20
13827/13827 [==============================] - 13s - loss: 0.6704    
Epoch 6/20
13827/13827 [==============================] - 13s - loss: 0.6684    
Epoch 7/20
13827/13827 [==============================] - 13s - loss: 0.6669    
Epoch 8/20
13827/13827 [==============================] - 13s - loss: 0.6654    
Epoch 9/20
13827/13827 [==============================] - 13s - loss: 0.6642    
Epoch 10/20
13827/13827 [==============================] - 12s - loss: 0.6631    
Epoch 11/20
13827/13827 [==============================] - 13s - loss: 0.6622    
Epoch 12/20
13827/13827 [==============================] - 13s - loss: 0.6611    
Epoch 13/20
13827/13827 [

In [22]:
def classifier(d):
    num_to_remove = 3
    for i in range(num_to_remove):
        d.pop()
    d.add(Dense(128, input_dim=64, activation='tanh', bias=True))
    d.add(Dense(128, activation='tanh', bias=True))
    d.add(Dense(num_classes, activation='softmax', bias=True))
    d.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    return d

In [23]:
c = classifier(d)

In [24]:
c.fit(train_X, train_y, validation_data=(val_X, val_y), nb_epoch=nb_epochs, batch_size=batch_size)

Train on 13827 samples, validate on 6103 samples
Epoch 1/20
13827/13827 [==============================] - 10s - loss: 0.3211 - acc: 0.8862 - val_loss: 0.2488 - val_acc: 0.9135
Epoch 2/20
13827/13827 [==============================] - 10s - loss: 0.1381 - acc: 0.9518 - val_loss: 0.1407 - val_acc: 0.9512
Epoch 3/20
13827/13827 [==============================] - 10s - loss: 0.1066 - acc: 0.9634 - val_loss: 0.1823 - val_acc: 0.9451
Epoch 4/20
13827/13827 [==============================] - 10s - loss: 0.0977 - acc: 0.9666 - val_loss: 0.1467 - val_acc: 0.9536
Epoch 5/20
13827/13827 [==============================] - 11s - loss: 0.0710 - acc: 0.9761 - val_loss: 0.1584 - val_acc: 0.9508
Epoch 6/20
13827/13827 [==============================] - 11s - loss: 0.0768 - acc: 0.9739 - val_loss: 0.1314 - val_acc: 0.9623
Epoch 7/20
13827/13827 [==============================] - 11s - loss: 0.0790 - acc: 0.9743 - val_loss: 0.1361 - val_acc: 0.9585
Epoch 8/20
13827/13827 [==============================]

In [27]:
loss, acc = c.evaluate(test_AP_features, test_labels)

1111/1111 [==============================] - 0s     


In [28]:
print loss, acc

1.53804068433 0.714671467361
